In [1]:
from aloe.program import AloeProgram
text = """
modeh(*,daughter(+person,-person)).
modeb(*,parent(+person,-person)).
modeb(*,parent(-person,+person)).
modeb(*,female(+person)).
modeb(*,female(-person)).
%
determination(daughter/2,parent/2).
determination(daughter/2,female/1).
%    
set(verbose,0).

:- begin_bg.
person(ann).
person(mary).
person(tom).
person(eve).
person(lucy).

parent(ann,mary).
parent(ann,tom).
parent(tom,eve).
parent(tom,lucy).

female(ann).
female(mary).
female(eve).
female(lucy).
:- end_bg.
    

:- begin_in_pos.
daughter(lucy,tom).
daughter(mary,ann).
daughter(eve,tom).
:- end_in_pos.

:- begin_in_neg.
daughter(tom,ann).
daughter(tom,eve).
:- end_in_neg.
"""
ap = AloeProgram.build_from_text(text)
ap.induce()
#ap.query('female(X),parent(ann,X),person(Y).')


{B: 0, A: 1, C: 1, E: 2, D: 2}


LogicProgram object
daughter(A,B):-female(A).

In [2]:
ap.query('not(parent(X,ann)).')

(True, [{}])

In [3]:
from aloe.parser import AloeParser
parser = AloeParser()
parser.parse_query('not(parent(X,tom)).')

not(parent(X,tom))

In [1]:
from aloe.program import AloeProgram
from aloe.options import SystemParameters
import time
ap2 = AloeProgram.build_from_file("aloe/Examples/family.pl")
ap2.set('verbose',0)
t0 = time.time()
SystemParameters.generic_name_for_variable = True
out = ap2.induce()
t1 = time.time()
t1-t0

1.2144737243652344

In [2]:
ap2.knowledge

LogicProgram object
   person(andrew).
   person(bernard).
   person(cathleen).
   person(daphne).
   person(edith).
   person(fred).
   person(george).
   person(john).
   person(louis).
   person(oscar).
   person(paul).
   person(robert).
   person(stephen).
   person(sylvia).
   person(william).
   person(ada).
   father_of(william,sylvia).
   father_of(oscar,louis).
   father_of(oscar,daphne).
   father_of(oscar,cathleen).
   father_of(oscar,fred).
   father_of(oscar,bernard).
   father_of(louis,stephen).
   father_of(louis,andrew).
   father_of(louis,robert).
   father_of(louis,john).
   father_of(george,oscar).
   father_of(paul,edith).
   mother_of(sylvia,stephen).
   mother_of(sylvia,andrew).
   mother_of(sylvia,robert).
   mother_of(sylvia,john).
   mother_of(edith,louis).
   mother_of(edith,daphne).
   mother_of(edith,cathleen).
   mother_of(edith,fred).
   mother_of(edith,bernard).
   mother_of(ada,sylvia).
   parent_of(A,B):-mother_of(A,B).
   parent_of(A,B):-father_of(A,B

In [3]:
ap2.verify('grandparent_of(A,B):-grandfather_of(A,B).')

not(grandfather_of(A,B),not(grandparent_of(A,B)))


(True, [{}])

In [4]:
ap2.verify('grandfather_of(A,B):-grandparent_of(A,B).')

not(grandparent_of(A,B),not(grandfather_of(A,B)))


(False, [])

In [7]:
from aloe.knowledge import MultipleKnowledge, LogicProgram
ap = AloeProgram.build_from_text(text)
ap.knowledge = MultipleKnowledge(LogicProgram(options=ap.options), ap.knowledge, options=ap.options)
print(ap.query('female(mary).'))



h 0
Atom female(mary)
Candidates {female(mary).}
Match {female(mary).}
Clause female(mary).
Substitution {}
Atoms []
[{}]


In [2]:
female(X), not(parent(X,Y)), female(Y)

TypeError: object of type 'NoneType' has no len()

In [1]:
from aloe.clause import Clause, Predicate, Function, Constant, Variable, Operator
def print_class_in_depth(element):
    print(element.__class__, element)
    if isinstance(element, Operator):
        for t in element:
            print_class_in_depth(t)

atom = Predicate('female', [Constant('mary')])
print_class_in_depth(atom)            

<class 'aloe.clause.Predicate'> female(mary)
<class 'aloe.clause.Constant'> mary


In [10]:
def fun(a, b, **options):
    print(*options)
    
l = [1,2]
fun(*l, h=30, t=50)


h t


In [3]:
import aloe.program

t = """
mortel(X):-man(X).
mortel(X):-animal(X).

man(socrates).
animal(lion).
"""

ap = aloe.program.AloeProgram.build_from_background(t)

q = "mortel(X),not(man(X))."
ap.query(q)


(True, [{X: lion}])

In [5]:
a = {1,2,3}
b = {4,5}
c = {3,4}
a-c

{1, 2}

In [4]:
set.union({1,2},{3})

{1, 2, 3}

In [1]:
def fun():
    print(__name__)
    
fun()

__main__


In [1]:
from aloe.options import Options
class A:
    def __init__(self, options=None):
        self._options = options if options else Options()
        self._temp_options = dict()
        
    def __getattr__(self, attr):
        if attr=='options':
            if attr in self._temp_options:
                return self._temp_options[attr]
            else:
                return self._options, attr)
        else: return super().__getattr__(self)


In [3]:
a = A()
a.options.a

AttributeError: 'Options' object has no attribute 'options'

In [6]:
a.__dir__()

['_options',
 '_temp_options',
 '__module__',
 '__init__',
 '__getattr__',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [2]:
class A:
    def __init__(self, s: str):
        self.s = s
a = A(1)

In [4]:
a = {1}
isinstance(a , list)



False

In [8]:
help(iterable)

Help on class tuple in module builtins:

class tuple(object)
 |  tuple(iterable=(), /)
 |  
 |  Built-in immutable sequence.
 |  
 |  If no argument is given, the constructor returns an empty tuple.
 |  If iterable is specified the tuple is initialized from iterable's items.
 |  
 |  If the argument is a tuple, the return value is the same object.
 |  
 |  Built-in subclasses:
 |      asyncgen_hooks
 |      UnraisableHookArgs
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(self, /)
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __

In [12]:
raise NotImplementedError()

NotImplementedError: 

In [2]:
'a b \n c'.split()

['a', 'b', 'c']

In [3]:
a = #qmdfk
a

SyntaxError: invalid syntax (Temp/ipykernel_2252/2001087664.py, line 1)

In [2]:
any(True for _ in range(3))

True

In [3]:
isinstance((3,1),(int,int))

False

In [8]:
class SubstitutionError(Exception):
    def __init__(self, key, item):
        message = 'Substitution {%s/%s} is not valid' % (str(key), str(item))
        super().__init__(message)
        
raise SubstitutionError(3,4)

SubstitutionError: Substitution {3/4} is not valid

In [14]:
import os

def switch_Function_CompoundTerm(fname):
    f = open(fname, 'r')
    t = f.read()
    f.close()

    t = t.replace("Function", "FFFFF")
    t = t.replace("CompoundTerm", "Function")
    t = t.replace("FFFFF", "CompoundTerm")

    f = open(fname, 'w')
    f.write(t)
    f.close()

    
    
"""
directory = "aloe"
# iterate over files in
# that directory
for filename in os.listdir(directory):
    fname = os.path.join(directory, filename)
    if os.path.isfile(fname) and fname[-3:]==".py":
        switch_Function_CompoundTerm(fname)

"""

'\ndirectory = "aloe"\n# iterate over files in\n# that directory\nfor filename in os.listdir(directory):\n    fname = os.path.join(directory, filename)\n    if os.path.isfile(fname) and fname[-3:]==".py":\n        switch_Function_CompoundTerm(fname)\n\n'

In [3]:
"""
import os
directory = "aloe"
# iterate over files in
# that directory
for filename in os.listdir(directory):
    fname = os.path.join(directory, filename)
    if os.path.isfile(fname) and fname[-3:]==".py":
        f = open(fname, 'r')
        t = f.read()
        f.close()

        t = t.replace("ModeHandler", "ModeCollection")

        f = open(fname, 'w')
        f.write(t)
        f.close()        
"""

In [2]:
from itertools import chain

In [3]:
'a'.capitalize

Help on class chain in module itertools:

class chain(builtins.object)
 |  chain(*iterables) --> chain object
 |  
 |  Return a chain object whose .__next__() method returns elements from the
 |  first iterable until it is exhausted, then elements from the next
 |  iterable, until all of the iterables are exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __setstate__(...)
 |      Set state information for unpickling.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __class_getitem__(...) from builtins.type
 |      See PEP 585
 |  
 |  from_iterable(iterable, /) from builtins.type
 |      Alternative chain() constructor taking a single iterable argument tha

In [5]:
class A:
    def __init__(self):
        self.l = [1,2,3]
        
    def __iter__(self):
        return iter(self.l)
    
a = A()
list(a)

[1, 2, 3]

In [19]:
s = 'mqkdm'
s.capitalize()

'Mqkdm'

In [11]:
def fun(x):
    if x:
        yield x
        
g = fun(0)


SyntaxError: unexpected EOF while parsing (Temp/ipykernel_10752/2609936294.py, line 7)

In [16]:
ord('A')

65

In [17]:
for a in range('A','Z'):
    print(a)

TypeError: 'str' object cannot be interpreted as an integer

In [2]:
class A:
    a = 5
    
A.a = 6
a = A()
a.a

6

In [6]:
s = """
ada parent_of andrew.
ada parent_of john.
ada parent_of robert.
ada parent_of stephen.
edith parent_of andrew.
edith parent_of john.
edith parent_of robert.
edith parent_of stephen.
george parent_of bernard.
george parent_of cathleen.
george parent_of daphne.
george parent_of fred.
george parent_of louis.
oscar parent_of andrew.
oscar parent_of john.
oscar parent_of robert.
oscar parent_of stephen.
paul parent_of bernard.
paul parent_of cathleen.
paul parent_of daphne.
paul parent_of fred.
paul parent_of louis.
william parent_of andrew.
william parent_of john.
william parent_of robert.
william parent_of stephen.

sylvia parent_of ada.
bernard parent_of edith.
cathleen parent_of edith.
daphne parent_of edith.
fred parent_of edith.
louis parent_of edith.
oscar parent_of george.
andrew parent_of louis.
john parent_of louis.
robert parent_of louis.
stephen parent_of louis.
bernard parent_of oscar.
cathleen parent_of oscar.
daphne parent_of oscar.
fred parent_of oscar.
louis parent_of oscar.
edith parent_of paul.
andrew parent_of sylvia.
john parent_of sylvia.
robert parent_of sylvia.
stephen parent_of sylvia.
sylvia parent_of william.
ada grandparent_of sylvia.

edith grandparent_of bernard.
edith grandparent_of cathleen.
edith grandparent_of daphne.
edith grandparent_of fred.
edith grandparent_of louis.
george grandparent_of oscar.
louis grandparent_of andrew.
louis grandparent_of john.
louis grandparent_of robert.
louis grandparent_of stephen.
oscar grandparent_of bernard.
oscar grandparent_of cathleen.
oscar grandparent_of daphne.
oscar grandparent_of fred.
oscar grandparent_of louis.
paul grandparent_of edith.
sylvia grandparent_of andrew.
sylvia grandparent_of john.
sylvia grandparent_of robert.
sylvia grandparent_of stephen.
william grandparent_of sylvia.

ada grandfather_of sylvia.
edith grandfather_of bernard.
edith grandfather_of cathleen.
edith grandfather_of daphne.
edith grandfather_of fred.
edith grandfather_of louis.
george grandfather_of oscar.
louis grandfather_of andrew.
louis grandfather_of john.
louis grandfather_of robert.
louis grandfather_of stephen.
oscar grandfather_of bernard.
oscar grandfather_of cathleen.
oscar grandfather_of daphne.
oscar grandfather_of fred.
oscar grandfather_of louis.
paul grandfather_of edith.
sylvia grandfather_of andrew.
sylvia grandfather_of john.
sylvia grandfather_of robert.
sylvia grandfather_of stephen.
william grandfather_of sylvia.

ada grandfather_of andrew.
ada grandfather_of john.
ada grandfather_of robert.
ada grandfather_of stephen.
edith grandfather_of andrew.
edith grandfather_of john.
edith grandfather_of robert.
edith grandfather_of stephen.
"""
s = s.replace(".","")
out = ''
for line in s.split("\n"):
    if line: 
        a, p, b, *garbage = line.split(' ')
        out += '%s(%s,%s).' % (p, a, b)
    out += '\n'

print(out)


parent_of(ada,andrew).
parent_of(ada,john).
parent_of(ada,robert).
parent_of(ada,stephen).
parent_of(edith,andrew).
parent_of(edith,john).
parent_of(edith,robert).
parent_of(edith,stephen).
parent_of(george,bernard).
parent_of(george,cathleen).
parent_of(george,daphne).
parent_of(george,fred).
parent_of(george,louis).
parent_of(oscar,andrew).
parent_of(oscar,john).
parent_of(oscar,robert).
parent_of(oscar,stephen).
parent_of(paul,bernard).
parent_of(paul,cathleen).
parent_of(paul,daphne).
parent_of(paul,fred).
parent_of(paul,louis).
parent_of(william,andrew).
parent_of(william,john).
parent_of(william,robert).
parent_of(william,stephen).

parent_of(sylvia,ada).
parent_of(bernard,edith).
parent_of(cathleen,edith).
parent_of(daphne,edith).
parent_of(fred,edith).
parent_of(louis,edith).
parent_of(oscar,george).
parent_of(andrew,louis).
parent_of(john,louis).
parent_of(robert,louis).
parent_of(stephen,louis).
parent_of(bernard,oscar).
parent_of(cathleen,oscar).
parent_of(daphne,oscar).
pa

In [3]:
help(tuple)

Help on class tuple in module builtins:

class tuple(object)
 |  tuple(iterable=(), /)
 |  
 |  Built-in immutable sequence.
 |  
 |  If no argument is given, the constructor returns an empty tuple.
 |  If iterable is specified the tuple is initialized from iterable's items.
 |  
 |  If the argument is a tuple, the return value is the same object.
 |  
 |  Built-in subclasses:
 |      asyncgen_hooks
 |      UnraisableHookArgs
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(self, /)
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __

In [5]:
from aloe.utils import generate_variable_names
t0 = time.time()
generate_variable_names()
time.time()-t0

0.0

In [4]:
help(list)

Help on class list in module builtins:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self))

In [1]:
class A:
    def __init__(self):
        self.a = 3
        self.b = 4
    def fun(self,c):
        self.c = c
        
a = A()
t = a.fun
t(5)
print(a.c)
t(6)
print(a.c)

5
6


In [3]:
class Inf(int):
    def __init__(self)

a = A(1)
a

1

In [10]:
class A(list):
    pass

b = A((1,))
a = A((1,)) + A((3,4)) 
a.__class__

list